<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate some random data for demonstration purposes
np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(2, size=(1000, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(X_train.shape[1],), activation='linear'),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # For binary classification
    # If you have more than two classes, use 'softmax' activation and adjust the number of units accordingly
    # tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# For multi-class classification, use 'categorical_crossentropy' as the loss function

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
25/25 [==============================] - 5s 77ms/step - loss: 0.6955 - accuracy: 0.5175 - val_loss: 0.6981 - val_accuracy: 0.4550
Epoch 2/10
25/25 [==============================] - 0s 17ms/step - loss: 0.6908 - accuracy: 0.5188 - val_loss: 0.6970 - val_accuracy: 0.4950
Epoch 3/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6919 - accuracy: 0.5200 - val_loss: 0.6987 - val_accuracy: 0.4550
Epoch 4/10
25/25 [==============================] - 0s 17ms/step - loss: 0.6916 - accuracy: 0.5150 - val_loss: 0.6940 - val_accuracy: 0.5250
Epoch 5/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6898 - accuracy: 0.5263 - val_loss: 0.6986 - val_accuracy: 0.4650
Epoch 6/10
25/25 [==============================] - 0s 18ms/step - loss: 0.6897 - accuracy: 0.5100 - val_loss: 0.6961 - val_accuracy: 0.4900
Epoch 7/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6890 - accuracy: 0.5387 - val_loss: 0.6967 - val_accuracy: 0.4750
Epoch 8/10
25